In [24]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import tensorflow.keras as keras
from tensorflow.keras.optimizers.legacy import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from keras import regularizers

In [2]:
DATASET_PATH = "oye.json"

In [3]:
def load_data(dataset_path):
    with open(dataset_path, "r") as fp:
        data = json.load(fp)
        
        # convert lists into numpy arrays
        inputs = np.array(data["mfcc"])
        targets = np.array(data["labels"])
        
        return inputs, targets

In [4]:
# Set up K-Fold Cross-Validation
k = 5  # Number of folds
kf = KFold(n_splits=k, shuffle=True, random_state=42)

In [25]:
def create_model(inputs):
    
    # build the network architecture
    model = keras.Sequential([
        # input layer
        keras.layers.Flatten(input_shape=(inputs.shape[1], inputs.shape[2])),
        
        # 1st hidden layer
        keras.layers.Dense(512, activation="relu",kernel_regularizer=keras.regularizers.l2(0.001)),
        keras.layers.Dropout(0.2),

        
        # 2nd hidden layer
        keras.layers.Dense(128, activation="relu"),
        
        # 3nd hidden layer
        keras.layers.Dense(64, activation="relu"),
        
        # output layer
        keras.layers.Dense(11, activation="softmax")
    ])
    
    # compile network
    optimizer = Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer,
                 loss="sparse_categorical_crossentropy",
                 metrics=["accuracy"]
                 )
    
    return model

In [26]:
if __name__ == "__main__":
    inputs, targets = load_data(DATASET_PATH)
    
    # List to store accuracy for each fold
    accuracy_per_fold = []
    
    # split data into train and test sets
    inputs_train, inputs_test, targets_train, targets_test = train_test_split(inputs, targets, test_size=0.3)

      # K-Fold Cross-Validation
    for fold, (train_index, val_index) in enumerate(kf.split(inputs_train)):
        print(f'Fold {fold + 1}')

                # Split the data into training and validation sets for this fold
        X_train_fold, X_val_fold = inputs_train[train_index], inputs_train[val_index]
        y_train_fold, y_val_fold = targets_train[train_index], targets_train[val_index]

                # Create a new instance of the model for each fold
        model = create_model(inputs_train)

                # Train the model on the training fold
        model.fit(X_train_fold, y_train_fold, epochs=5, batch_size=32, verbose=0)

                # Evaluate the model on the validation fold
        val_predictions = np.argmax(model.predict(X_val_fold), axis=1)
        accuracy = accuracy_score(y_val_fold, val_predictions)

                # Store the accuracy for this fold
        accuracy_per_fold.append(accuracy)
        print(f'Accuracy for fold {fold + 1}: {accuracy * 100:.2f}%')

    # Calculate the average accuracy across all folds
    average_accuracy = np.mean(accuracy_per_fold)
    print(f'\nAverage Accuracy Across {k} Folds: {average_accuracy * 100:.2f}%')

Fold 1
44/44 [==============================] - 0s 1ms/step
Accuracy for fold 1: 39.63%
Fold 2
44/44 [==============================] - 0s 2ms/step
Accuracy for fold 2: 39.70%
Fold 3
44/44 [==============================] - 0s 997us/step
Accuracy for fold 3: 32.33%
Fold 4
44/44 [==============================] - 0s 1ms/step
Accuracy for fold 4: 29.97%
Fold 5
44/44 [==============================] - 0s 1ms/step
Accuracy for fold 5: 41.63%

Average Accuracy Across 5 Folds: 36.65%


In [19]:
if __name__ == "__main__":
    inputs, targets = load_data(DATASET_PATH)
    
    # split data into train and test sets
    inputs_train, inputs_test, targets_train, targets_test = train_test_split(inputs, targets, test_size=0.4)
    
    model = create_model(inputs)
    
    model.summary()
    
    print("Inputs train shape:", inputs_train.shape)
    print("Targets train shape:", targets_train.shape)
    print("Inputs test shape:", inputs_test.shape)
    print("Targets test shape:", targets_test.shape)

    
    #train network
    model.fit(x=inputs_train, 
              y=targets_train,
             validation_data=(inputs_test, targets_test),
             epochs=50,
             batch_size=32
             )
    

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_17 (Flatten)        (None, 1690)              0         
                                                                 
 dense_68 (Dense)            (None, 256)               432896    
                                                                 
 dropout_5 (Dropout)         (None, 256)               0         
                                                                 
 dense_69 (Dense)            (None, 128)               32896     
                                                                 
 dense_70 (Dense)            (None, 64)                8256      
                                                                 
 dense_71 (Dense)            (None, 11)                715       
                                                                 
Total params: 474,763
Trainable params: 474,763
Non-t

Epoch 49/50
188/188 [==============================] - 1s 3ms/step - loss: 1.4221 - accuracy: 0.4861 - val_loss: 1.5930 - val_accuracy: 0.4461
Epoch 50/50
188/188 [==============================] - 1s 4ms/step - loss: 1.3959 - accuracy: 0.4884 - val_loss: 1.5821 - val_accuracy: 0.4553
